# FIRST TRY

ora proviamo, con tutto ciò che fino ad ora abbiamo implementato, a sviluppare qualcosa.

plausibile che useremo come primo esperimento niente meno che "jump", il gioco del dinosauro di google.

In [4]:
import numpy as np
import keras
import random
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


usiamo (copia ed incollando) la rescaled probability function

In [2]:
def target_linear_rescaled_distribution(reward, royal_action, starting_distribution):

    assert reward >= -1 and reward <= 1, 'reward must be between -1 and 1'

    starting_distribution = np.array(starting_distribution)
    coefficient = np.zeros(len(starting_distribution))
    new_royal_probability = min(1, starting_distribution[royal_action]*(1 + reward))
    coefficient[royal_action] = new_royal_probability # this is not the real coefficient we use it to be quicker

    if new_royal_probability == 1:
        return coefficient #just a trick to be quicker in case we hit a 1
    
    rest = starting_distribution[royal_action]*reward
    sum_others = 1 - starting_distribution[royal_action]
    ratio = rest/sum_others

    coefficient = np.ones(len(starting_distribution))*(1 - ratio)
    coefficient[royal_action] = (1 + reward)
    
    return starting_distribution*coefficient.T

la custom network (anch'essa copia ed incollata)

In [3]:
class CustomModel(keras.Model):
    def __init__(self, input_size, number_of_hidden, output_size, hidden_size, activ = "relu", out_activ = "softmax"):
        super().__init__()
        self.input1 = keras.Input(shape=(input_size,)) # gli input sono 4
        self.hidden = []
        for i in range (number_of_hidden):
            self.hidden.append(keras.layers.Dense(hidden_size[i], activation = activ))
        self.output1 = keras.layers.Dense(output_size, activation = out_activ)

    def call(self, inputs):
        x = inputs
        for h in self.hidden:
            x = h(x)
        return self.output1(x)

ed ora definiamo il gioco

In [5]:
import pygame
import random

# Inizializzazione di pygame
pygame.init()

# Dimensioni della finestra
width, height = 800, 500
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("player Jump")

# Colori
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)

# Parametri del player
player_width, player_height = 50, 50
player_x = 50
player_y = height - player_height - 20
player_velocity = 20
gravity = 1
jump_strength = -20

ground = height - 20

obstacle_width, obstacle_height = 20, 40
obstacle_speed = 10
obstacle_frequency = 1500  # millisecondi tra l'apparizione di un nuovo ostacolo

# Inizializza il clock
clock = pygame.time.Clock()

# Lista degli ostacoli
obstacles = []
spawn_time = 0

# Variabili del player
is_jumping = False
player_y_velocity = 0

# Costruzione del player

player = CustomModel()

# Funzione per disegnare il playersauro
def draw_player(x, y):
    pygame.draw.rect(win, black, (x, y, player_width, player_height))

# Funzione per disegnare gli ostacoli
def draw_obstacle(obstacle):
    pygame.draw.rect(win, red, obstacle)

# Funzione per controllare le collisioni
def check_collision(player_rect, obstacles):
    for obstacle in obstacles:
        if player_rect.colliderect(obstacle):
            return True
    return False

# Funzione per mostrare il punteggio
def draw_score(win, score, font_size=36):
    font = pygame.font.SysFont(None, font_size)
    score_text = font.render(f"Score: {score}", True, (0, 0, 0))
    text_rect = score_text.get_rect(center=(width // 2, 20))
    win.blit(score_text, text_rect)

fps = 60
score = 0

# Loop principale
running = True
while running:
    clock.tick(fps)
    win.fill(white)
    score += 1

    draw_score(win, score, font_size=36)

    # Gestione eventi
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Controllo del salto
    keys = pygame.key.get_pressed()
    # keys = player.choose(env)
    if keys[pygame.K_SPACE] and not is_jumping:
        is_jumping = True
        player_y_velocity = jump_strength

    # Movimento del playersauro
    if is_jumping:
        player_y += player_y_velocity
        player_y_velocity += gravity
        if player_y >= ground - player_height:
            player_y = ground - player_height
            is_jumping = False

    # Gestione degli ostacoli
    current_time = pygame.time.get_ticks()
    if current_time - spawn_time > obstacle_frequency:
        obstacle_x = width
        obstacle_y = ground - obstacle_height
        obstacles.append(pygame.Rect(obstacle_x, obstacle_y, obstacle_width, obstacle_height))
        spawn_time = current_time
        obstacle_frequency = random.randint(800, 1500)
        obstacle_speed += 0.1

    # Movimento degli ostacoli
    for obstacle in obstacles:
        obstacle.x -= obstacle_speed
        draw_obstacle(obstacle)

    # Rimozione degli ostacoli fuori schermo
    obstacles = [obstacle for obstacle in obstacles if obstacle.x + obstacle_width > 0]

    # Controllo delle collisioni
    player_rect = pygame.Rect(player_x, player_y, player_width, player_height)
    if check_collision(player_rect, obstacles):
        print("Hai perso!")
        running = False

    # Disegna il playersauro
    draw_player(player_x, player_y)

    # Aggiorna la finestra
    pygame.display.update()

pygame.quit()

Hai perso!
